### Attemping to make an animated matplotlib celldevs plot
See this links.
- http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
- https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.load.html
- https://www.numpy.org/devdocs/reference/generated/numpy.lib.format.html
- https://github.com/rogersce/cnpy

In [54]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10,8)
import matplotlib.animation as animation
from IPython.display import HTML

In [44]:
%load_ext autoreload
%autoreload 2
from pycdevs.cdppwrapper import CDPPWrapper, Model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
cellDevsShape = (40, 40)
modelSourceCode = """
[Top]
components : voronoiExpansion

[voronoiExpansion]
type : cell
dim : (${spaceHeight},${spaceWidth})
delay : transport
defaultDelayTime  : 0
border : nowrapped
neighbors : voronoiExpansion(-1,-1) voronoiExpansion(-1,0) voronoiExpansion(-1,1)
neighbors : voronoiExpansion(0,-1)  voronoiExpansion(0,0)  voronoiExpansion(0,1)
neighbors : voronoiExpansion(1,-1)  voronoiExpansion(1,0)  voronoiExpansion(1,1)
initialvalue : 0
initialCellsValue : sites.val
localtransition : voronoiSite

[voronoiSite]

% Expand center cell value in a 3x3 square neighbourhood
rule : {(1,0)} ${expansionWaveDelay} { (0,0)=0 and (1,0)!=0 and (1,0)!=? }
rule : {(1,-1)} ${expansionWaveDelay} { (0,0)=0 and (1,-1)!=0  and  (1,-1)!=? }
rule : {(0,-1)} ${expansionWaveDelay} { (0,0)=0 and (0,-1)!=0  and  (0,-1)!=? }
rule : {(-1,-1)} ${expansionWaveDelay} { (0,0)=0 and (-1,-1)!=0  and  (-1,-1)!=? }
rule : {(-1,0)} ${expansionWaveDelay} { (0,0)=0 and (-1,0)!=0  and  (-1,0)!=? }
rule : {(-1,1)} ${expansionWaveDelay} { (0,0)=0 and (-1,1)!=0  and  (-1,1)!=? }
rule : {(1,1)} ${expansionWaveDelay} { (0,0)=0 and (1,1)!=0  and  (1,1)!=? }
rule : {(0,1)} ${expansionWaveDelay} { (0,0)=0 and (0,1)!=0  and  (0,1)!=? }

% Default rule
rule : {(0,0)} 0 { t }
"""
# TODO: Make some kind of meta-DSL for replacements and formulaes inside CellDEVS model DSL
modelSourceCode = modelSourceCode\
    .replace('${expansionWaveDelay}', str(100))\
    .replace('${spaceHeight}', str(cellDevsShape[0]))\
    .replace('${spaceWidth}', str(cellDevsShape[1]))
simpleVoronoiModel = Model(modelSourceCode, 'voronoiExpansion')

In [46]:
# TODO: VTime builder wrapper
wrapper = CDPPWrapper(simpleVoronoiModel, '00:01:00:00')
wrapper.run()

In [47]:
drawlogNPFilename = wrapper.drawlog('00:00:00:100')

In [48]:
loadedMatrix = np.load(drawlogNPFilename)
# loadedMatrix = loadedMatrix.reshape(cellDevsShape)

composedMatrix = np.zeros([cellDevsShape[0], cellDevsShape[1], len(loadedMatrix.files)], dtype=np.double)

for index, npFileName in enumerate(loadedMatrix.files):
    composedMatrix[..., index] = loadedMatrix[ npFileName ].reshape(cellDevsShape)

In [55]:
%%capture
index = 0
fig, ax = plt.subplots()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plottedImage = ax.imshow(composedMatrix[..., index])
stepsCount = composedMatrix.shape[2]

In [56]:
def updateImage(*args):
    global index
    index += 1
    index = index % stepsCount
    plottedImage.set_array(composedMatrix[..., index])
    return plottedImage,
    
simulationAnimation = animation.FuncAnimation(fig, updateImage, interval= 100)

HTML(simulationAnimation.to_html5_video())